In [79]:
import os
import math
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
from matplotlib import cm
from scipy import stats
from scipy import optimize
import numpy.polynomial.polynomial as poly
from bisect import bisect_left

In [80]:
def import_file(path, limit_low=None, limit_high=None):

    spectrum = np.genfromtxt(path, delimiter=",")
    spectrum = np.transpose(spectrum)
    wavenumbers = spectrum[0]
    intensities = spectrum[1]

    if limit_low is not None:
        limit_low_index = list(wavenumbers).index(limit_low)
    else:
        limit_low_index = 0
        limit_low = wavenumbers[0]

    if limit_high is not None:
        limit_high_index = list(wavenumbers).index(limit_high)
    else:
        limit_high_index = len(wavenumbers)
        limit_high = wavenumbers[-1]

    wavenumbers = wavenumbers[limit_low_index:limit_high_index]
    intensities = intensities[limit_low_index:limit_high_index]
    return wavenumbers, intensities

def import_directory(path, limit_low=None, limit_high=None):
    # files = os.listdir(path)

    # for filename in files:
    #     np.genfromtxt(filename, delimiter=",")
    pass

In [81]:
wavenumbers, intensities = import_file("spectra/E (1).TXT", 600, 995)

In [82]:
intensities_sg0 = sig.savgol_filter(intensities, 
                                    window_length=9, 
                                    polyorder=3, 
                                    deriv=0)

intensities_sg1 = sig.savgol_filter(intensities, 
                                    window_length=9, 
                                    polyorder=3, 
                                    deriv=1)

intensities_sg2 = sig.savgol_filter(intensities, 
                                    window_length=9, 
                                    polyorder=3, 
                                    deriv=2)

intensities_sg3 = sig.savgol_filter(intensities, 
                                    window_length=9, 
                                    polyorder=3, 
                                    deriv=3)

In [83]:
zeros_sg3 = np.diff(np.sign(intensities_sg3))
peaks = np.where(zeros_sg3 > 0)[0]
peaks = peaks[intensities_sg2[peaks] / intensities_sg0[peaks] < 0]

peak_condensing = []
peaks_new = []
for i in range(len(wavenumbers)-1):
    if i in peaks:
        peak_condensing.append(i)
    if intensities_sg2[i] > 0 and len(peak_condensing) > 0:
        peaks_new.append(int(np.mean(peak_condensing)))
        peak_condensing = []
if len(peak_condensing) > 0:
    peaks_new.append(int(np.mean(peak_condensing)))
peaks_new = np.asarray(peaks_new)            

In [84]:
## Basislinie Versuch 1

zeros_sg1 = np.diff(np.sign(intensities_sg1))
mins = list(np.where(zeros_sg1 > 0)[0])
mins.insert(0, 0)
mins.append(len(wavenumbers)-1)
intervals = {}
for i in range(len(mins)-1):
    interval = (mins[i], mins[i+1])
    peaks = peaks_new[np.where(np.logical_and(peaks_new > interval[0], peaks_new < interval[1]))]
    if len(peaks) > 0:
        if i == 0:
            baseline_points = [0] + mins[:3]
            baseline_wns = wavenumbers[baseline_points]
            baseline_wns[0] = 2*baseline_wns[1] - baseline_wns[2]
        elif i == len(mins)-2:
            baseline_points = mins[-3:] + [mins[-1]]
            baseline_wns = wavenumbers[baseline_points]
            baseline_wns[-1] = 2*baseline_wns[-2] - baseline_wns[-3]
        else:
            baseline_points = mins[i-1:i+3]
            baseline_wns = wavenumbers[baseline_points]
        baseline_ints = intensities_sg0[baseline_points]
        interval_relative = [i-baseline_points[0] for i in interval]
        fit = poly.polyfit(baseline_wns, baseline_ints, 3)
        x = wavenumbers[baseline_points[0]:baseline_points[-1]]
        ints = intensities_sg0[baseline_points[0]:baseline_points[-1]]
        ints_sg2 = intensities_sg2[baseline_points[0]:baseline_points[-1]]
        local_baseline = poly.polyval(x, fit)
        ints_corrected = ints - local_baseline
        peaks = peaks - baseline_points[0]

        valleys = list(np.asarray(interval) - baseline_points[0])
        for i in range(len(peaks)-1):
            if not any(j in valleys for j in range(peaks[i], peaks[i+1])):
                valleys.append(peaks[i] + np.argmax(ints_sg2[peaks[i]:peaks[i+1]]))

        valleys.sort() 
        
        print(valleys)
        
        
        intervals[interval] = {"wavenumbers":x,
                               "intensities":ints,
                               "baseline_ints":local_baseline,
                               "corrected_ints":ints_corrected,
                               "peaks":peaks,
                               "valleys":valleys}


[0, 81, 195]
[195, 267, 347]
[152, 187]
[35, 58]
[23, 103]
[80, 110, 185]
[105, 153, 175]
[70, 149, 199]


In [85]:
## Basislinie Versuch 2

zeros_sg1 = np.diff(np.sign(intensities_sg1))
mins = list(np.where(zeros_sg1 > 0)[0])
mins.insert(0, 0)
mins.append(len(wavenumbers)-1)
intervals = {}
for i in range(len(mins)-1):
    interval = (mins[i], mins[i+1])
    print(interval)
    peaks = peaks_new[np.where(np.logical_and(peaks_new > interval[0], peaks_new < interval[1]))]
    if len(peaks) > 0:
        wns = [wavenumbers[i] for i in interval]
        baseline_ints = [intensities_sg0[i] for i in interval]
        interval_relative = [0, interval[1]-interval[0]-1]
        fit = poly.polyfit(wns, baseline_ints, 1)
        x = wavenumbers[interval[0]:interval[1]]
        ints = intensities_sg0[interval[0]:interval[1]]
        ints_sg2 = intensities_sg2[interval[0]:interval[-1]]
        local_baseline = poly.polyval(x, fit)
        ints_corrected = ints - local_baseline
        peaks = peaks - interval[0]

        valleys = list(interval_relative)
        for i in range(len(peaks)-1):
            if not any(j in valleys for j in range(peaks[i], peaks[i+1])):
                valleys.append(peaks[i] + np.argmax(ints_sg2[peaks[i]:peaks[i+1]]))

        valleys.sort() 
        
        print(valleys)
        
        
        intervals[interval] = {"wavenumbers":x,
                               "intensities":ints,
                               "baseline_ints":local_baseline,
                               "corrected_ints":ints_corrected,
                               "peaks":peaks,
                               "valleys":valleys}


(0, 195)
[0, 81, 194]
(195, 347)
[0, 72, 151]
(347, 382)
[0, 34]
(382, 405)
[0, 22]
(405, 485)
[0, 79]
(485, 590)
[0, 30, 104]
(590, 660)
[0, 48, 69]
(660, 789)
[0, 79, 128]


In [89]:
## Basislinie Versuch 3 

# Liste an Minima erstellen
zeros_sg1 = np.diff(np.sign(intensities_sg1))
mins = list(np.where(zeros_sg1 > 0)[0])
mins.insert(0, 0)
mins.append(len(wavenumbers)-1)

intervals = {}
for i in range(len(mins)-1):
    interval = (mins[i], mins[i+1]) # Immer 2 benachbarte Minima sind ein Intervall
    
    #Ein Minimum davor und danach hinzufügen -> für Polynom
    if i == 0:
        baseline_points = [0] + mins[:3]
        baseline_wns = wavenumbers[baseline_points]
        baseline_wns[0] = 2*baseline_wns[1] - baseline_wns[2]
    elif i == len(mins)-2:
        baseline_points = mins[-3:] + [mins[-1]]
        baseline_wns = wavenumbers[baseline_points]
        baseline_wns[-1] = 2*baseline_wns[-2] - baseline_wns[-3]
    else:
        baseline_points = mins[i-1:i+3]
        baseline_wns = wavenumbers[baseline_points]
    baseline_ints = intensities_sg0[baseline_points]
    
    # Polynom fitten und Basislinie berechnen
    interval_relative = [i-baseline_points[0] for i in interval]
    fit = poly.polyfit(baseline_wns, baseline_ints, 3)
    x = wavenumbers[baseline_points[0]:baseline_points[-1]]
    local_baseline = poly.polyval(x, fit)
    
    wns = wavenumbers[interval[0]:interval[1]]
    ints = intensities_sg0[interval[0]:interval[1]]
    ints_sg2 = intensities_sg2[interval[0]:interval[1]]
    ints_corrected = ints - local_baseline[interval_relative[0]:interval_relative[1]]
    
    # Negative Bereiche korrigieren
    mins_new = sig.argrelmin(ints_corrected)[0]
    
    mins_new = [0] + [x for x in mins_new if ints_corrected[x] < 0] + [len(ints_corrected)-1]
    
    baseline2_wns = wns[mins_new]
    baseline2_ints = ints_corrected[mins_new]
    fit2 = poly.polyfit(baseline2_wns, baseline2_ints, len(mins_new)-1)
    local_baseline2 = poly.polyval(x, fit2)
    ints_corrected = ints_corrected - local_baseline2[interval_relative[0]:interval_relative[1]]
    local_baseline += local_baseline2
    
    intervals[interval] = {"wavenumbers":wns,
                           "intensities":ints,
                           "ints_sg2":ints_sg2,
                           "baseline_wns":x,
                           "baseline_ints":local_baseline,
                           "corrected_ints":ints_corrected,
                           "rel_int":interval_relative}


In [109]:
%matplotlib
for i, d in intervals.items():
    fig, ax = plt.subplots(2,1)
    ax[0].plot(d["wavenumbers"], d["intensities"], linewidth=1, color="blue")
    ax[0].plot(d["baseline_wns"], d["baseline_ints"], linewidth=1, color="red")
    ax[0].fill_between(d["wavenumbers"],
                       d["intensities"],
                       d["baseline_ints"][d["rel_int"][0]:d["rel_int"][1]])
    ax[1].plot(d["wavenumbers"], d["corrected_ints"], linewidth=1, color="blue")
    ax[1].hlines(0, d["wavenumbers"][0], d["wavenumbers"][-1])
    ax[1].fill_between(d["wavenumbers"],
                       d["corrected_ints"],
                       0)
    
    #ax.vlines(values["wavenumbers"][values["peaks"]], np.min(values["intensities"])*0.9, np.max(values["intensities"])*1.1, color="red")
    #ax.vlines(values["wavenumbers"][values["valleys"]], np.min(values["intensities"])*0.9, np.max(values["intensities"])*1.1, color="blue")

Using matplotlib backend: Qt5Agg


In [108]:
## Peakfinding

for i, d in intervals.items():
    peaks = sig.argrelmin(d["ints_sg2"], order=10)[0]
    peaks = [peak for peak in peaks if d["ints_sg2"][peak] < 0]
    peaks_height = d["corrected_ints"][peaks]
    
    peaks = [peak for _,peak in sorted(zip(peaks_height,peaks))]
    peaks.reverse()
    peaks_wns = d["wavenumbers"][peaks]
    peaks_height = d["corrected_ints"][peaks]
    
    print(i)
    print(peaks_wns)
    print(peaks_height)
    print("")
    

(0, 195)
[660.5 621. ]
[11429.87981027  1134.59780083]

(195, 347)
[724.  744.5]
[5105.24964658 3429.6822411 ]

(347, 382)
[779.5]
[64.29588074]

(382, 405)
[798.5]
[32.23246323]

(405, 485)
[825.]
[639.60865545]

(485, 590)
[872.  848.5]
[502.12723719 199.49006363]

(590, 660)
[905.5 924.5]
[285.73765861  69.23913585]

(660, 789)
[958.5 985.5]
[2826.06648757  215.62142985]



In [96]:
def gaussian(x, amplitude, mean, stddev):
    return (amplitude * np.exp(-((x - mean) / 4 / stddev)**2))

Signature: sig.argrelmin(data, axis=0, order=1, mode='clip')
Docstring:
Calculate the relative minima of `data`.

Parameters
----------
data : ndarray
    Array in which to find the relative minima.
axis : int, optional
    Axis over which to select from `data`. Default is 0.
order : int, optional
    How many points on each side to use for the comparison
    to consider ``comparator(n, n+x)`` to be True.
mode : str, optional
    How the edges of the vector are treated.
    Available options are 'wrap' (wrap around) or 'clip' (treat overflow
    as the same as the last (or first) element).
    Default 'clip'. See numpy.take.

Returns
-------
extrema : tuple of ndarrays
    Indices of the minima in arrays of integers. ``extrema[k]`` is
    the array of indices of axis `k` of `data`. Note that the
    return value is a tuple even when `data` is 1-D.

See Also
--------
argrelextrema, argrelmax, find_peaks

Notes
-----
This function uses `argrelextrema` with np.less as comparator. Therefor

In [12]:
def gaussian(x, amplitude, mean, stddev, offset):
    return (amplitude * np.exp(-((x - mean) / 4 / stddev)**2)) + offset

for interval, values in intervals.items():
    print(interval)
    intervals[interval]["parameters"] = []
    intervals[interval]["gauss_ints"] = []
    for i in range(len(values["peaks"])):
        lower_bound = values["valleys"][i]
        upper_bound = values["valleys"][i+1]
        peak = values["peaks"][i]
        range_wn = values["wavenumbers"][upper_bound] - values["wavenumbers"][lower_bound]
        range_ints = np.max(values["corrected_ints"]) - np.min(values["corrected_ints"])
        initial_guess = (values["corrected_ints"][peak], values["wavenumbers"][peak], values["wavenumbers"][upper_bound]-values["wavenumbers"][lower_bound], 0)
        print(initial_guess)
        parameters = optimize.curve_fit(gaussian, 
                                        values['wavenumbers'][lower_bound:upper_bound], 
                                        values['corrected_ints'][lower_bound:upper_bound], 
                                        p0=initial_guess, 
                                        bounds=[(0, values["wavenumbers"][lower_bound], 0, -range_ints*1.5), (range_ints*1.5, values["wavenumbers"][upper_bound], range_wn, range_ints*1.5)])[0]
        print(parameters)
        intervals[interval]["parameters"].append(parameters)
        gauss_ints = []
        for x in values["wavenumbers"]:
            gauss_ints.append(gaussian(x, parameters[0], parameters[1], parameters[2], parameters[3]))
        intervals[interval]["gauss_ints"].append(np.asarray(gauss_ints))
            

(0, 97)


KeyError: 'peaks'

In [ ]:
%matplotlib
for interval, values in intervals.items():
    print(interval)
    fig, ax = plt.subplots()
    ax.plot(values["wavenumbers"], values["corrected_ints"], linewidth=1, color="blue")
    ax.plot(values["wavenumbers"], values["gauss_ints"][0])
    ax.vlines(values["wavenumbers"][values["peaks"]], 0, np.max(values["corrected_ints"])*1.1, color="red")
    ax.vlines(values["wavenumbers"][values["valleys"]], 0, np.max(values["corrected_ints"])*1.1, color="blue")

In [ ]:
?sig.argrelmin

In [ ]:
def plot_sg(wavenumbers, intensities0, intensities1, intensities2, intensities3):
    fig, ax = plt.subplots(2,2)
    ax[0,0].plot(wavenumbers, intensities0, linewidth = 1)
    ax[0,0].plot(x, local_baseline, linewidth = 1, color="red")
    ax[0,0].set_title("Spektrum")
    ax[0,1].plot(wavenumbers, intensities1, linewidth = 1)
    ax[0,1].set_title("1. Ableitung")
    ax[1,0].plot(wavenumbers, intensities2, linewidth = 1)
    ax[1,0].set_title("2. Ableitung")
    ax[1,1].plot(wavenumbers, intensities3, linewidth = 1)
    ax[1,1].set_title("3. Ableitung")
    major_ticks = ticker.MultipleLocator(100)
    minor_ticks = ticker.MultipleLocator(20)
    for i, plot in np.ndenumerate(ax):
        ax[i].set_xlim(wavenumbers[0], wavenumbers[-1])
        ax[i].invert_xaxis()
        ax[i].xaxis.set_major_locator(major_ticks)
        ax[i].xaxis.set_minor_locator(minor_ticks)
        ax[i].grid()
        if i != 0:
            ax[i].hlines(0, wavenumbers[0], wavenumbers[-1], linewidth=1, color="black")
    ax[0,0].vlines(wavenumbers[peaks_new], 0, np.max(intensities0)*1.1, linewidth=0.7, color="red")
    ax[0,1].vlines(wavenumbers[peaks_new], np.min(intensities1), np.max(intensities1)*1.1, linewidth=0.7, color="red")
    ax[1,0].vlines(wavenumbers[peaks_new], np.min(intensities2), np.max(intensities2)*1.1, linewidth=0.7, color="red")
    ax[0,0].vlines(wavenumbers[mins[1:-1]], 0, np.max(intensities)*1.1, linewidth=0.7, color="blue")
    ax[0,0].set_ylim(np.min(intensities)*0.9, np.max(intensities)*1.1)

In [ ]:
%matplotlib
plot_sg(wavenumbers, intensities_sg0, intensities_sg1, intensities_sg2, intensities_sg3)

In [ ]:
valleys = list(mins)
for i in range(len(peaks_new)-1):
    if not any(j in mins for j in range(peaks_new[i], peaks_new[i+1])):
        valleys.append(peaks_new[i] + np.argmax(intensities_sg2[peaks_new[i]:peaks_new[i+1]]))

valleys.sort()    
    

In [ ]:
type(peaks)

In [ ]:
?optimize.curve_fit